--- Day 2: Rock Paper Scissors ---

The Elves begin to set up camp on the beach. To decide whose tent gets to be closest to the snack storage, a giant Rock Paper Scissors tournament is already in progress.

Rock Paper Scissors is a game between two players. Each game contains many rounds; in each round, the players each simultaneously choose one of Rock, Paper, or Scissors using a hand shape. Then, a winner for that round is selected: Rock defeats Scissors, Scissors defeats Paper, and Paper defeats Rock. If both players choose the same shape, the round instead ends in a draw.

Appreciative of your help yesterday, one Elf gives you an encrypted strategy guide (your puzzle input) that they say will be sure to help you win. "The first column is what your opponent is going to play: A for Rock, B for Paper, and C for Scissors. The second column--" Suddenly, the Elf is called away to help with someone's tent.

The second column, you reason, must be what you should play in response: X for Rock, Y for Paper, and Z for Scissors. Winning every time would be suspicious, so the responses must have been carefully chosen.

The winner of the whole tournament is the player with the highest score. Your total score is the sum of your scores for each round. The score for a single round is the score for the shape you selected (1 for Rock, 2 for Paper, and 3 for Scissors) plus the score for the outcome of the round (0 if you lost, 3 if the round was a draw, and 6 if you won).

Since you can't be sure if the Elf is trying to help you or trick you, you should calculate the score you would get if you were to follow the strategy guide.

For example, suppose you were given the following strategy guide:
```
A Y
B X
C Z
```
This strategy guide predicts and recommends the following:

    In the first round, your opponent will choose Rock (A), and you should choose Paper (Y). This ends in a win for you with a score of 8 (2 because you chose Paper + 6 because you won).
    In the second round, your opponent will choose Paper (B), and you should choose Rock (X). This ends in a loss for you with a score of 1 (1 + 0).
    The third round is a draw with both players choosing Scissors, giving you a score of 3 + 3 = 6.

In this example, if you were to follow the strategy guide, you would get a total score of 15 (8 + 1 + 6).

What would your total score be if everything goes exactly according to your strategy guide?


In [5]:
// Data Types and Functions
public enum Choice {
    Rock,
    Paper,
    Scissors,
}

Choice ParseChoice(string choice) {
    return choice switch {
        "A" or "X" => Choice.Rock,
        "B" or "Y" => Choice.Paper,
        "C" or "Z" => Choice.Scissors,
        _ => throw new System.InvalidOperationException("Unknown choice"),
    };
}

public record struct Part1Input {
    public Choice OpponentChoice;
    public Choice MyChoice;
}

Part1Input ParseLinePart1(string line) {
    var sections = line.Split(" ");
    return new Part1Input {
        OpponentChoice = ParseChoice(sections[0]),
        MyChoice = ParseChoice(sections[1]),
    };
}

public enum MatchOutcome {
    Win,
    Draw,
    Loss
}

MatchOutcome ParseMatchOutcome(string choice) {
    return choice switch {
        "X" => MatchOutcome.Loss,
        "Y" => MatchOutcome.Draw,
        "Z" => MatchOutcome.Win,
        _ => throw new System.InvalidOperationException("Unknown match outcome"),
    };
}

public record struct Part2Input {
    public Choice OpponentChoice;
    public MatchOutcome DesiredOutcome;
}

Part2Input ParseLinePart2(string line) {
    var sections = line.Split(" ");
    return new Part2Input {
        OpponentChoice = ParseChoice(sections[0]),
        DesiredOutcome = ParseMatchOutcome(sections[1]),
    };
}

public record struct MatchResult {
    public MatchOutcome Outcome;
    public Choice MyChoice;
}

int Score(MatchResult result) {
    var pickScore = result.MyChoice switch {
        Choice.Rock => 1,
        Choice.Paper => 2,
        Choice.Scissors => 3,
    };
    var outcomeScore = result.Outcome switch {
        MatchOutcome.Win => 6,
        MatchOutcome.Draw => 3,
        MatchOutcome.Loss => 0,
    };
    return pickScore + outcomeScore;
}

MatchResult PlayGame(Choice opponents, Choice mine) {
    var outcome = (opponents, mine) switch {
        (Choice.Rock, Choice.Rock) => MatchOutcome.Draw,
        (Choice.Rock, Choice.Paper) => MatchOutcome.Win,
        (Choice.Rock, Choice.Scissors) => MatchOutcome.Loss,

        (Choice.Paper, Choice.Paper) => MatchOutcome.Draw,
        (Choice.Paper, Choice.Scissors) => MatchOutcome.Win,
        (Choice.Paper, Choice.Rock) => MatchOutcome.Loss,

        (Choice.Scissors, Choice.Scissors) => MatchOutcome.Draw,
        (Choice.Scissors, Choice.Rock) => MatchOutcome.Win,
        (Choice.Scissors, Choice.Paper) => MatchOutcome.Loss,
    };
    return new MatchResult {
        Outcome = outcome,
        MyChoice = mine,
    };
}

Choice ChoiceNeededForOutcome(Choice opponentChoice, MatchOutcome outcome) {
    return (opponentChoice, outcome) switch {
        (Choice.Rock, MatchOutcome.Win) => Choice.Paper,
        (Choice.Rock, MatchOutcome.Draw) => Choice.Rock,
        (Choice.Rock, MatchOutcome.Loss) => Choice.Scissors,

        (Choice.Paper, MatchOutcome.Win) => Choice.Scissors,
        (Choice.Paper, MatchOutcome.Draw) => Choice.Paper,
        (Choice.Paper, MatchOutcome.Loss) => Choice.Rock,

        (Choice.Scissors, MatchOutcome.Win) => Choice.Rock,
        (Choice.Scissors, MatchOutcome.Draw) => Choice.Scissors,
        (Choice.Scissors, MatchOutcome.Loss) => Choice.Paper,
    };
}

In [6]:
// Read input
var lines = System.IO.File.ReadAllLines("input.txt");
lines

index,value
0,A X
1,A Z
2,A Z
3,A X
4,A X
5,B X
6,C Y
7,B X
8,A Z
9,A X


In [7]:
//Part 1
lines.Select(ParseLinePart1)
    .Select(choices => PlayGame(choices.OpponentChoice, choices.MyChoice))
    .Select(Score)
    .Sum()

8890

--- Part Two ---

The Elf finishes helping with the tent and sneaks back over to you. "Anyway, the second column says how the round needs to end: X means you need to lose, Y means you need to end the round in a draw, and Z means you need to win. Good luck!"

The total score is still calculated in the same way, but now you need to figure out what shape to choose so the round ends as indicated. The example above now goes like this:

    In the first round, your opponent will choose Rock (A), and you need the round to end in a draw (Y), so you also choose Rock. This gives you a score of 1 + 3 = 4.
    In the second round, your opponent will choose Paper (B), and you choose Rock so you lose (X) with a score of 1 + 0 = 1.
    In the third round, you will defeat your opponent's Scissors with Rock for a score of 1 + 6 = 7.

Now that you're correctly decrypting the ultra top secret strategy guide, you would get a total score of 12.

Following the Elf's instructions for the second column, what would your total score be if everything goes exactly according to your strategy guide?


In [8]:
// Part 2
lines.Select(ParseLinePart2)
    .Select(input => (input.OpponentChoice, ChoiceNeededForOutcome(input.OpponentChoice, input.DesiredOutcome)))
    .Select(choices => PlayGame(choices.Item1, choices.Item2))
    .Select(Score)
    .Sum()

10238